In [1]:
import os
import pickle
import pandas as pd
import json
from bag_of_words import bag_of_words_basic
from greedy_selection import greedy_selection_basic
from date_detection import date_detection_basic
from contextlib import redirect_stdout

/Users/ongsici/anaconda3/envs/peft_mia/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Pile-RS-Truncated

In [ ]:
def run_blind_mia_pile(pile_subset_list, seq_lens, attack, dataset_type, results):
    for pile_subset in pile_subset_list:
        base_dataset_dir = f"../../data/pile/pile_rs_truncated/{pile_subset}"
        output_dir = f"bow_results/{pile_subset}" if attack == "bow" else f"greedy_results/{pile_subset}" if attack == "greedy" else f"date_results/{pile_subset}"
        os.makedirs(output_dir, exist_ok=True)
        results[attack][pile_subset] = {}

        for seq_len in seq_lens:
            results[attack][pile_subset][seq_len] = {}
            dataset_path = f"{base_dataset_dir}/pile_{dataset_type}_{pile_subset}_seqlen{seq_len}.pkl"
            output_path = f"pile/{output_dir}/pile_{dataset_type}_{pile_subset}_seqlen{seq_len}_results.txt"
            
            if not os.path.exists(dataset_path):
                print(f"Skipping {dataset_path} (not found)")
                continue
            
            with open(dataset_path, "rb") as f:
                df = pickle.load(f)

            X = df['truncated_text'].astype(str).tolist()
            y = df['label'].astype(int).tolist()
            members = df[df['label'] == 1]['truncated_text'].astype(str).tolist()
            nonmembers = df[df['label'] == 0]['truncated_text'].astype(str).tolist()

            dataset_name = f"pile_{dataset_type}_{pile_subset}_seqlen{seq_len}"
            fpr_budget = 1
            plot_roc = False
            hypersearch = True

            with open(output_path, "w") as fout:
                with redirect_stdout(fout):
                    print(f"Running bag-of-words for seq_len={seq_len}")
                    if attack == "bow":
                        mean_auc, mean_auc_stdev, mean_tpr, mean_tpr_stdev = bag_of_words_basic(X, y, dataset_name, fpr_budget, plot_roc, hypersearch)
                        results[attack][pile_subset][seq_len]["mean_auc"] = mean_auc
                        results[attack][pile_subset][seq_len]["mean_auc_stdev"] = mean_auc_stdev
                        results[attack][pile_subset][seq_len]["mean_tpr"] = mean_tpr
                        results[attack][pile_subset][seq_len]["mean_tpr_stdev"] = mean_tpr_stdev
                    elif attack == "greedy":
                        mean_auc, mean_auc_stdev, mean_tpr, mean_tpr_stdev = greedy_selection_basic(member_lines=members, nonmember_lines=nonmembers, dataset_name=dataset_name, fpr_budget=fpr_budget, plot_roc=plot_roc)
                        results[attack][pile_subset][seq_len]["mean_auc"] = mean_auc
                        results[attack][pile_subset][seq_len]["mean_auc_stdev"] = mean_auc_stdev
                        results[attack][pile_subset][seq_len]["mean_tpr"] = mean_tpr
                        results[attack][pile_subset][seq_len]["mean_tpr_stdev"] = mean_tpr_stdev
                    elif attack == "date":
                        max_auc, max_tpr = date_detection_basic(X, y, dataset_name, fpr_budget, plot_roc)
                        results[attack][pile_subset][seq_len]["max_auc"] = max_auc
                        results[attack][pile_subset][seq_len]["max_tpr"] = max_tpr
                    else:
                        raise ValueError(f"Unknown attack type: {attack}")
                    
    return results



In [ ]:
seq_lens = [64, 128, 256, 512, 1024, 2048]
pile_subset_list = ["wiki", "pubmedc", "pilecc", "uspto", "github"]
dataset_type = "reservoirv2"


In [ ]:
## Date Detection
results = {}
attack = "date"
results[attack] = {}

results = run_blind_mia_pile(pile_subset_list, seq_lens, attack, dataset_type, results)

with open("pile/date_results.json", "w") as f:
    json.dump(results, f, indent=4)


In [ ]:
## Bag-of-Words Detection
results = {}
attack = "bow"
results[attack] = {}

results = run_blind_mia_pile(pile_subset_list, seq_lens, attack, dataset_type, results)

with open("pile/bow_results.json", "w") as f:
    json.dump(results, f, indent=4)


In [ ]:
## Greedy Selection
results = {}
attack = "greedy"
results[attack] = {}

results = run_blind_mia_pile(pile_subset_list, seq_lens, attack, dataset_type, results)

with open("pile/greedy_results.json", "w") as f:
    json.dump(results, f, indent=4)



## Dolma and Paloma

In [ ]:
def run_blind_mia_olmo(olmo_subset_list, seq_lens, attack, dataset_type, results):
    for olmo_subset in olmo_subset_list:
        base_dataset_dir = f"../../data/{dataset_type}/{olmo_subset}"
        output_dir = f"bow_results/{olmo_subset}" if attack == "bow" else f"greedy_results/{olmo_subset}" if attack == "greedy" else f"date_results/{olmo_subset}"
        os.makedirs(output_dir, exist_ok=True)
        results[attack][olmo_subset] = {}

        for seq_len in seq_lens:
            results[attack][olmo_subset][seq_len] = {}
            dataset_path = f"{base_dataset_dir}/{olmo_subset}_seqlen{seq_len}.pkl"
            output_path = f"{dataset_type}/{output_dir}/{olmo_subset}_seqlen{seq_len}_results.txt"
            
            # Skip if dataset doesn't exist
            if not os.path.exists(dataset_path):
                print(f"Skipping {dataset_path} (not found)")
                continue
            
            with open(dataset_path, "rb") as f:
                df = pickle.load(f)

            X = df['truncated_text'].astype(str).tolist()
            y = df['label'].astype(int).tolist()
            members = df[df['label'] == 1]['truncated_text'].astype(str).tolist()
            nonmembers = df[df['label'] == 0]['truncated_text'].astype(str).tolist()

            dataset_name = f"pile_{dataset_type}_{olmo_subset}_seqlen{seq_len}"
            fpr_budget = 1
            plot_roc = False
            hypersearch = True

            with open(output_path, "w") as fout:
                with redirect_stdout(fout):
                    print(f"Running bag-of-words for seq_len={seq_len}")
                    if attack == "bow":
                        mean_auc, mean_auc_stdev, mean_tpr, mean_tpr_stdev = bag_of_words_basic(X, y, dataset_name, fpr_budget, plot_roc, hypersearch)
                        results[attack][olmo_subset][seq_len]["mean_auc"] = mean_auc
                        results[attack][olmo_subset][seq_len]["mean_auc_stdev"] = mean_auc_stdev
                        results[attack][olmo_subset][seq_len]["mean_tpr"] = mean_tpr
                        results[attack][olmo_subset][seq_len]["mean_tpr_stdev"] = mean_tpr_stdev
                    elif attack == "greedy":
                        mean_auc, mean_auc_stdev, mean_tpr, mean_tpr_stdev = greedy_selection_basic(member_lines=members, nonmember_lines=nonmembers, dataset_name=dataset_name, fpr_budget=fpr_budget, plot_roc=plot_roc)
                        results[attack][olmo_subset][seq_len]["mean_auc"] = mean_auc
                        results[attack][olmo_subset][seq_len]["mean_auc_stdev"] = mean_auc_stdev
                        results[attack][olmo_subset][seq_len]["mean_tpr"] = mean_tpr
                        results[attack][olmo_subset][seq_len]["mean_tpr_stdev"] = mean_tpr_stdev
                    elif attack == "date":
                        max_auc, max_tpr = date_detection_basic(X, y, dataset_name, fpr_budget, plot_roc)
                        results[attack][olmo_subset][seq_len]["max_auc"] = max_auc
                        results[attack][olmo_subset][seq_len]["max_tpr"] = max_tpr
                    else:
                        raise ValueError(f"Unknown attack type: {attack}")
    return results

In [ ]:
seq_lens = [64, 128, 256, 512, 1024, 2048]
olmo_subset_list = ["wiki", "stack", "pes2o"]
dataset_type = "olmo"

In [18]:
## Date Detection
results = {}
attack = "date"
results[attack] = {}

results = run_blind_mia_olmo(olmo_subset_list, seq_lens, attack, dataset_type, results)

with open("olmo_date_results.json", "w") as f:
    json.dump(results, f, indent=4)


In [ ]:
## BoW Detection
results = {}
attack = "bow"
results[attack] = {}

results = run_blind_mia_olmo(olmo_subset_list, seq_lens, attack, dataset_type, results)

with open("olmo/bow_results.json", "w") as f:
    json.dump(results, f, indent=4)


In [ ]:
## Greedy Selection
results = {}
attack = "greedy"
results[attack] = {}

results = run_blind_mia_olmo(olmo_subset_list, seq_lens, attack, dataset_type, results)

with open("olmo/greedy_results.json", "w") as f:
    json.dump(results, f, indent=4)
